In [53]:
!python -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1
    Uninstalling pip-25.1:
      Successfully uninstalled pip-25.1


In [54]:

!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to c:\users\hp\appdata\local\temp\pip-req-build-ynhsbnh5
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Hp\AppData\Local\Temp\pip-req-build-ynhsbnh5'


Defaulting to user installation because normal site-packages is not writeable


In [55]:
import os
import json
from jiwer import wer
import whisper
import torchaudio
import torchaudio.transforms as T  
import re
import traceback

In [56]:
!pip install jiwer

Defaulting to user installation because normal site-packages is not writeable


In [57]:
audio_directory=r"C:\Users\Hp\Desktop\audios"
sentences_path=r"C:\Users\Hp\Desktop\sentences.json"  
output_directory=r"C:\Users\Hp\Desktop\stt_output"
os.makedirs(output_directory, exist_ok=True)

In [6]:
model=whisper.load_model("large")

C:\Users\Hp\AppData\Roaming\Python\Python312\site-packages\whisper\__init__.py:69: UserWarning: C:\Users\Hp\.cache\whisper\large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [22:42<00:00, 2.27MiB/s]


In [58]:
valid_extensions=['.wav']
audio_files=[f for f in os.listdir(audio_directory) if os.path.splitext(f)[1].lower() in valid_extensions]

print(f"Found {len(audio_files)} audio files:")
for f in audio_files:
    print(f)

Found 100 audio files:
common_voice_ps_10115310722352403962771918791058.wav
common_voice_ps_10160918354791458783187858551036.wav
common_voice_ps_102286703511560267744063597681026.wav
common_voice_ps_1023583642114419588918588034991006.wav
common_voice_ps_102684632989313487016492462981078.wav
common_voice_ps_1079263116128335853118163176831100.wav
common_voice_ps_117546820568446714210963011771014.wav
common_voice_ps_1213329694151177786514753313201041.wav
common_voice_ps_12341691618347315615542201581056.wav
common_voice_ps_1235978420192613770714481331601073.wav
common_voice_ps_1242365521151494341215750622971065.wav
common_voice_ps_132894342212580736891241195961045.wav
common_voice_ps_14063706628943969918790007431027.wav
common_voice_ps_140861711019371824572785953631069.wav
common_voice_ps_14161438370651579973885481052.wav
common_voice_ps_14170547891867791843426828411070.wav
common_voice_ps_14225035038050896955486525421068.wav
common_voice_ps_143654770515200444592157541761077.wav
common_voi

In [59]:
for file in audio_files:
    file_path = os.path.join(audio_directory, file)
    size = os.path.getsize(file_path)
    if size == 0:
        print(f"File '{file}' is empty (0 bytes).")
    else:
        print(f"File '{file}' is {size} bytes.")

File 'common_voice_ps_10115310722352403962771918791058.wav' is 261628 bytes.
File 'common_voice_ps_10160918354791458783187858551036.wav' is 206642 bytes.
File 'common_voice_ps_102286703511560267744063597681026.wav' is 202186 bytes.
File 'common_voice_ps_1023583642114419588918588034991006.wav' is 107076 bytes.
File 'common_voice_ps_102684632989313487016492462981078.wav' is 112278 bytes.
File 'common_voice_ps_1079263116128335853118163176831100.wav' is 101876 bytes.
File 'common_voice_ps_117546820568446714210963011771014.wav' is 82556 bytes.
File 'common_voice_ps_1213329694151177786514753313201041.wav' is 248254 bytes.
File 'common_voice_ps_12341691618347315615542201581056.wav' is 280204 bytes.
File 'common_voice_ps_1235978420192613770714481331601073.wav' is 112278 bytes.
File 'common_voice_ps_1242365521151494341215750622971065.wav' is 246024 bytes.
File 'common_voice_ps_132894342212580736891241195961045.wav' is 93702 bytes.
File 'common_voice_ps_14063706628943969918790007431027.wav' is 1

In [60]:
def preprocess_audio(file_path):
    try:
        waveform, sr=torchaudio.load(file_path)
        waveform=waveform.mean(dim=0).unsqueeze(0) 
        resampler=T.Resample(orig_freq=sr, new_freq=16000)
        waveform=resampler(waveform)

        output_path=os.path.join(output_directory, "processed_" + os.path.splitext(os.path.basename(file_path))[0] + ".wav")
        os.makedirs(output_directory, exist_ok=True)
        print(f"Saving processed audio to: {output_path}")

        torchaudio.save(output_path, waveform, 16000, format="wav")
        print(f"✅ Processed audio saved successfully: {output_path}")
        return output_path

    except Exception as e:
        print(f"Error processing audio file '{file_path}': {e}")
        traceback.print_exc()
        return None


In [61]:
def normalize(text):
    text=text.lower()
    text=re.sub(r'[\W_]+', ' ', text) 
    return text.strip()

In [62]:
try:
    with open(sentences_path, "r", encoding="utf-8") as f:
        full_data=json.load(f)
    print("JSON loaded successfully.")
except json.JSONDecodeError as e:
    print(f"JSON Decode Error: {e}")
    exit()
except FileNotFoundError:
    print("File not found.")
    exit()

ground_truth = None
for item in full_data:
    if item.get("type")=="table" and item.get("name") == "sentences":
        ground_truth=item.get("data", [])
        break

if ground_truth is None:
    print("sentences table not found!")
    exit()
else:
    print(f"Found 'sentences' table with {len(ground_truth)} entries.")

try:
    ground_truth_dict={entry["sentence_id"]: entry["sentence"] for entry in ground_truth}
    print(f"Dictionary created with {len(ground_truth_dict)} key-value pairs.")
except KeyError as e:
    print(f"Missing expected key: {e}")
    exit()

for sid, sent in list(ground_truth_dict.items())[:10]:
    print(f"{sid}: {sent}")

JSON loaded successfully.
Found 'sentences' table with 100 entries.
Dictionary created with 100 key-value pairs.
1001: -	یو څه شیان مې په کار دي.
1002: -	ته یې را ته وایه زه به یې درته را اخلم.
1003: -	ته صبر دلته مې لیکلي دي:
1004: 1.	یودانه برس. 2.	څلور دانې صابونان. 3.	درې بوتله شامپو. 4.	دوه دانې د غاښونو کریم. 5.	یوه جوړه کالي.
1005: -	ښه دا خو ستا شیان شول.
1006: -	دا برس دانه په څو دی؟
1007: -	برس به په شل افغانۍ درته حساب کړم.
1008: -	صابون دانه په څو دی؟
1009: -	چې ستا مرعات په کښې وي، په شل افغانۍ.
1010: -	شامپو به نو په څو وي؟


In [49]:
#TRANSCRIPTION
results={}
predictions=[]
references = []

print("\n🎙️ Starting transcription...\n")

valid_extensions=['.wav']
audio_files=f for f in os.listdir(audio_directory) if os.path.splitext(f)[1].lower() in valid_extensions]

for filename in audio_files:
    file_path=os.path.join(audio_directory, filename)

    if os.path.getsize(file_path)==0:
        print(f"⚠️ Skipping empty file: {filename}")
        continue

    print(f"🎧 Processing: {filename}")
    processed_path=preprocess_audio(file_path)
    file_id = os.path.splitext(filename)[0].replace("common_voice_ps_", "")

    #Transcribing
    result=model.transcribe(processed_path, language='ps', task='transcribe', fp16=False)
    predicted_text=result['text'].strip()

    results[filename]=predicted_text
    with open(os.path.join(output_directory, filename + ".txt"), "w", encoding="utf-8") as f:
        f.write(predicted_text)

    expected_text=ground_truth_dict.get(file_id, "").strip()
    print(f"📝 Predicted : {predicted_text}")

    if expected_text:
        norm_pred=normalize(predicted_text)
        norm_ref=normalize(expected_text)
        predictions.append(norm_pred)
        references.append(norm_ref)
        error=wer(norm_ref, norm_pred)
        print(f"📊 WER: {round(error * 100, 2)}%")

    print("-" * 50)


🎙️ Starting transcription...

🎧 Processing: common_voice_ps_10115310722352403962771918791058.wav
Saving processed audio to: C:\Users\Hp\Desktop\stt_output\processed_common_voice_ps_10115310722352403962771918791058.wav
✅ Processed audio saved successfully: C:\Users\Hp\Desktop\stt_output\processed_common_voice_ps_10115310722352403962771918791058.wav
📝 Predicted : احمد پینزه قلمون لری. دلت پینزه عددی صفت تای دی قلمون و شمیری بیان کرده دهی.
--------------------------------------------------
🎧 Processing: common_voice_ps_10160918354791458783187858551036.wav
Saving processed audio to: C:\Users\Hp\Desktop\stt_output\processed_common_voice_ps_10160918354791458783187858551036.wav
✅ Processed audio saved successfully: C:\Users\Hp\Desktop\stt_output\processed_common_voice_ps_10160918354791458783187858551036.wav
📝 Predicted : دا هغزمیرون دی چی دی نوم پرزای دی پختن پر داول کاراولکی گی.
--------------------------------------------------
🎧 Processing: common_voice_ps_10228670351156026774406359768102

In [63]:
# === FINAL METRICS ===
if references and predictions:
    print("\n📈 Final Evaluation Across Dataset")
    print(f"Average WER: {round(wer(references, predictions) * 100, 2)}%")
    print(f"Average CER: {round(cer(references, predictions) * 100, 2)}%")

In [64]:
#VALIDATION 
print("-- Validating with additional Pashto audio file --\n")

#manually added a voice recording 
custom_audio_path=r"C:\Users\Hp\Downloads\um.wav"
manual_ground_truth="زا تا سرادیرا مینا م"  

processed_path=preprocess_audio(custom_audio_path)

result=model.transcribe(processed_path, language='ps', task='transcribe', fp16=False)
predicted_text=result['text'].strip()

print(f"Predicted : {predicted_text}")
print(f"Expected  : {manual_ground_truth}")

norm_pred=normalize(predicted_text)
norm_ref=normalize(manual_ground_truth)
error=wer(norm_ref, norm_pred)
print(f"WER: {round(error * 100, 2)}%")

-- Validating with additional Pashto audio file --

Saving processed audio to: C:\Users\Hp\Desktop\stt_output\processed_um.wav
✅ Processed audio saved successfully: C:\Users\Hp\Desktop\stt_output\processed_um.wav
Predicted : زا تا سرا دیرا مینا گم
Expected  : زا تا سرادیرا مینا م
WER: 60.0%
